In [29]:
all = [var for var in globals() if var[0] != '_']
for var in all:
    del globals()[var]
del var, all
# Clear all global variables except those starting with an underscore

In [30]:
import os, sys
import numpy as np
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Tuple, Set, Any
from scipy.sparse import lil_matrix, csr_matrix
from scipy.sparse.linalg import spsolve
import meshio

1. .msh 파일을 일종의 text 파일처럼 import / 이게 불가하면 먼저 import하고 text 데이터로 변환 후 변수로 data_msh라는 변수로 저장
2. dict_section이라는 이름의 빈 딕셔너리 하나 만들자.
3. 이 data_msh 변수를 대상으로, 첫 '\$'가 나오는 줄에서 '\$' 다음에 나오는 텍스트를 name_section라는 이름의 변수로 저장
4. 그 다음 줄부터를 탐색 대상으로 하여 '\$' + name_section 이라는 문자열이 나오는 줄을 찾고, 첫 '\$'이 나왔던 줄의 다음 줄부터 해당 줄 바로 앞줄까지의 데이터를 복사하여 data_section이라는 변수로 저장
5. 그리고 방금 찾은 첫 '\$'가 나오는 줄에서부터 '\$' + name_section 이라는 문자열이 나오는 줄까지를 data_msh 변수에서 삭제
6. dict_section에 key = name_section, value = data_section 으로 하여 저장
7. 다시 data_msh 변수를 대상으로 더 이상 data_msh에 텍스트가 남아있지 않을 때까지 3~5 반복
8. dict_section에서 key = PhysicalNames에 해당하는 value를 대상으로 "displacementBC"라는 텍스트가 들어있는 줄과 "loadBC"라는 텍스트가 들어있는 줄을 추출하여 하나의 변수(data_BC) 안에 저장
9. 8에서 추출된 BC가 적용될 PhysicalNames에 대해, 이게 몇 차원의 entity에 속한 건지, Tag는 몇인지 읽고, 해당 차원의 entity 데이터(dict_section의 Entities key에 해당하는 value)로 가서 각 entity들마다의 physicalTag를 읽으면서 해당 PhysicalNames Tag에 대응되는 entity의 Tag 확인하고 이걸 data_BC에서 대응되는 각 줄의 맨 끝에 추가
10. dict_section에서 key = 'Nodes'에 해당하는 value를 data_nodes라는 변수로 생성
11. dict_section에서 key = 'Elements'에 해당하는 value를 data_elements라는 변수로 생성
12. data_elements에서 data_BC의 각 BC의 entity 차원 및 Tag에 대응되는 element들 찾아서 걔네들의 node Tag만 따로 뺀 다음 중복 Tag는 하나만 담게 하여(unique filtering) data_BC의 대응되는 BC 줄의 우측에 추가
13. data_nodes라는 변수에서 nodes의 index, 글로벌 좌표만 나래비로 뽑아서 info_nodes라는 변수로 지정
14. data_elements라는 변수에서 elements의 type, index, 가지고 있는 node의 index만 나래비로 뽑아서 info_elements라는 변수로 지정

이 모든 걸 구현하는 코드 짜줘.

In [31]:
# ----------------------------
# 데이터 구조
# ----------------------------
@dataclass
class BCInfo:
    name: str                 # "displacementBC" / "loadBC"
    dim: int                  # physical dimension (e.g., 2 for surface)
    physical_tag: int         # physical group id
    entity_tag: int | None = None  # 해당 physical tag가 붙은 geometric entity tag (예: surface tag)
    node_tags: List[int] | None = None  # 해당 BC entity에 속한 element들의 node tag (unique)


# ----------------------------
# 1~7: 섹션 분해 로직 (요구사항대로 data_msh에서 잘라내며 dict_section에 저장)
# ----------------------------
def split_msh_sections_to_dict(path: str) -> Tuple[List[str], Dict[str, List[str]]]:
    """
    1) 파일을 텍스트로 읽어서 data_msh(list[str])로 저장
    2) dict_section 생성
    3~7) data_msh에서 $Section ... $EndSection 구간을 잘라 dict_section[name]=data_section으로 저장
         그리고 해당 구간은 data_msh에서 삭제
    """
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        data_msh = f.read().splitlines()

    dict_section: Dict[str, List[str]] = {}

    def find_first_section_start(lines: List[str]) -> int | None:
        for idx, line in enumerate(lines):
            s = line.strip()
            # 섹션 시작은 "$"로 시작하고 "$End"로 시작하지 않는 라인
            if s.startswith("$") and not s.startswith("$End"):
                return idx
        return None

    while True:
        i0 = find_first_section_start(data_msh)
        if i0 is None:
            break

        # 3) '$' 다음 텍스트를 name_section으로
        name_section = data_msh[i0].strip()[1:].strip()  # "$Nodes" -> "Nodes"
        end_marker = "$End" + name_section

        # 4) 다음 줄부터 end_marker 찾기
        i1 = None
        for j in range(i0 + 1, len(data_msh)):
            if data_msh[j].strip() == end_marker:
                i1 = j
                break
        if i1 is None:
            raise ValueError(f"섹션 {name_section}의 종료 마커({end_marker})를 찾지 못했습니다.")

        data_section = data_msh[i0 + 1 : i1]  # 시작 다음 줄부터 종료 바로 전까지

        # 5) data_msh에서 섹션 전체 삭제 (시작 줄 ~ 종료 줄)
        del data_msh[i0 : i1 + 1]

        # 6) dict_section 저장
        dict_section[name_section] = data_section

    return data_msh, dict_section


# ----------------------------
# 8: PhysicalNames에서 BC 라인 추출
# ----------------------------
def parse_physicalnames_for_bcs(physical_lines: List[str], bc_names: Tuple[str, str]) -> List[BCInfo]:
    """
    msh4 $PhysicalNames:
      <numPhysicalNames>
      <dim> <physicalTag> "name"
      ...
    """
    if not physical_lines:
        raise ValueError("$PhysicalNames 섹션이 비어 있습니다.")

    n = int(physical_lines[0].strip())
    out: List[BCInfo] = []
    targets = set(bc_names)

    for line in physical_lines[1:1+n]:
        s = line.strip()
        # 예: 2 2 "displacementBC"
        parts = s.split()
        if len(parts) < 3:
            continue
        dim = int(parts[0])
        physical_tag = int(parts[1])

        # name은 큰따옴표 포함. 공백 있을 수도 있으니 join 후 처리
        rest = " ".join(parts[2:])
        if '"' in rest:
            name = rest.strip()
            # "xxx" 형태에서 따옴표 제거
            if name.startswith('"') and name.endswith('"'):
                name = name[1:-1]
            else:
                # 예외적으로 따옴표가 중간에 있을 때
                name = name.replace('"', "")
        else:
            name = rest

        if name in targets:
            out.append(BCInfo(name=name, dim=dim, physical_tag=physical_tag))

    # 8) BC 두 개가 다 있는지 확인 (없어도 진행은 가능하게 하되, 경고)
    found = {b.name for b in out}
    missing = targets - found
    if missing:
        print(f"[경고] PhysicalNames에서 다음 BC 이름을 못 찾았습니다: {missing}")

    return out


# ----------------------------
# 9: Entities에서 physicalTag -> entityTag 매핑 찾기
# ----------------------------
def parse_entities_physical_map(entities_lines: List[str]) -> Dict[Tuple[int, int], Set[int]]:
    """
    msh4 $Entities:
      <numPoints> <numCurves> <numSurfaces> <numVolumes>
      (point blocks)
      (curve blocks)
      (surface blocks)
      (volume blocks)

    결과: (dim, entityTag) -> set(physicalTags)
    """
    if not entities_lines:
        raise ValueError("$Entities 섹션이 비어 있습니다.")

    header = entities_lines[0].split()
    if len(header) != 4:
        raise ValueError("$Entities 첫 줄 형식이 예상과 다릅니다.")
    npnt, ncrv, nsrf, nvol = map(int, header)

    idx = 1
    ent_phys: Dict[Tuple[int, int], Set[int]] = {}

    # ---- Points: pointTag x y z numPhysicalTags [physicalTags...]
    for _ in range(npnt):
        parts = entities_lines[idx].split()
        idx += 1
        point_tag = int(parts[0])
        num_phys = int(parts[4])
        phys_tags = set(map(int, parts[5:5+num_phys])) if num_phys > 0 else set()
        ent_phys[(0, point_tag)] = phys_tags

    # ---- Curves: curveTag bbox(6) numPhysicalTags [physicalTags...] numBoundingPoints pointTags...
    for _ in range(ncrv):
        parts = entities_lines[idx].split()
        idx += 1
        curve_tag = int(parts[0])
        num_phys = int(parts[7])
        p0 = 8
        phys_tags = set(map(int, parts[p0:p0+num_phys])) if num_phys > 0 else set()
        p1 = p0 + num_phys
        num_bpts = int(parts[p1])
        # bounding points는 p1+1 ~ p1+num_bpts (부호는 orientation)
        ent_phys[(1, curve_tag)] = phys_tags
        # (여기서는 bounding points 자체는 필요 없어서 저장 안 함)

    # ---- Surfaces: surfaceTag bbox(6) numPhysicalTags [physicalTags...] numBoundingCurves curveTags...
    for _ in range(nsrf):
        parts = entities_lines[idx].split()
        idx += 1
        srf_tag = int(parts[0])
        num_phys = int(parts[7])
        p0 = 8
        phys_tags = set(map(int, parts[p0:p0+num_phys])) if num_phys > 0 else set()
        p1 = p0 + num_phys
        num_bcrv = int(parts[p1])
        # bounding curves는 p1+1 ~ p1+num_bcrv (부호는 orientation)
        ent_phys[(2, srf_tag)] = phys_tags

    # ---- Volumes: volumeTag bbox(6) numPhysicalTags [physicalTags...] numBoundingSurfaces surfaceTags...
    for _ in range(nvol):
        parts = entities_lines[idx].split()
        idx += 1
        vol_tag = int(parts[0])
        num_phys = int(parts[7])
        p0 = 8
        phys_tags = set(map(int, parts[p0:p0+num_phys])) if num_phys > 0 else set()
        p1 = p0 + num_phys
        num_bsrf = int(parts[p1])
        ent_phys[(3, vol_tag)] = phys_tags

    return ent_phys


def attach_entity_tags_to_bcs(bcs: List[BCInfo], ent_phys_map: Dict[Tuple[int, int], Set[int]]) -> None:
    """
    9) 각 BC (dim, physical_tag)에 대해, 그 physical_tag를 포함하는 (dim, entity_tag)를 찾아 bc.entity_tag에 저장.
    """
    # dim별로 후보를 좁히면 빠름
    by_dim: Dict[int, List[Tuple[int, Set[int]]]] = {0: [], 1: [], 2: [], 3: []}
    for (dim, ent_tag), phys_tags in ent_phys_map.items():
        by_dim[dim].append((ent_tag, phys_tags))

    for bc in bcs:
        cand = by_dim.get(bc.dim, [])
        hits = [ent_tag for (ent_tag, phys_tags) in cand if bc.physical_tag in phys_tags]
        if not hits:
            bc.entity_tag = None
            print(f"[경고] BC '{bc.name}'(dim={bc.dim}, phys={bc.physical_tag})에 대응되는 entity_tag를 $Entities에서 못 찾았습니다.")
        elif len(hits) == 1:
            bc.entity_tag = hits[0]
        else:
            # 보통 1개여야 정상인데, 여러 엔티티에 같은 physical tag를 걸 수도 있어서
            # 여기서는 첫 번째만 붙이고 경고
            bc.entity_tag = hits[0]
            print(f"[경고] BC '{bc.name}'에 entity_tag 후보가 여러 개입니다: {hits}. 첫 번째({hits[0]})만 사용합니다.")


# ----------------------------
# 10~12: Nodes/Elements 파싱 + BC 요소에서 노드 유니크 추출
# ----------------------------
def parse_nodes_msh4(nodes_lines: List[str]) -> Tuple[np.ndarray, np.ndarray]:
    """
    msh4 $Nodes:
      numEntityBlocks totalNumNodes minTag maxTag
      (block 반복)
        entityDim entityTag parametric numNodesInBlock
        nodeTags (numNodesInBlock개, 여러 줄에 걸칠 수 있음)
        coords   (numNodesInBlock줄, x y z [parametric...])
    return:
      node_ids (N,)
      xyz (N,3)
    """
    if not nodes_lines:
        raise ValueError("$Nodes 섹션이 비어 있습니다.")

    num_blocks, total_nodes, min_tag, max_tag = map(int, nodes_lines[0].split())
    idx = 1

    node_ids: List[int] = []
    xyz: List[Tuple[float, float, float]] = []

    for _ in range(num_blocks):
        ent_dim, ent_tag, parametric, nblock = map(int, nodes_lines[idx].split())
        idx += 1

        # node tags (packed)
        tags: List[int] = []
        while len(tags) < nblock:
            tags.extend(map(int, nodes_lines[idx].split()))
            idx += 1

        # coords
        for k in range(nblock):
            parts = nodes_lines[idx].split()
            idx += 1
            x, y, z = map(float, parts[:3])
            node_ids.append(tags[k])
            xyz.append((x, y, z))
            # parametric 좌표는 무시

    node_ids_arr = np.array(node_ids, dtype=int)
    xyz_arr = np.array(xyz, dtype=float)

    # node_id 기준 정렬
    order = np.argsort(node_ids_arr)
    node_ids_arr = node_ids_arr[order]
    xyz_arr = xyz_arr[order]

    return node_ids_arr, xyz_arr


def parse_elements_msh4(elements_lines: List[str]) -> List[Tuple[int, int, List[int], int, int]]:
    """
    msh4 $Elements:
      numEntityBlocks totalNumElements minTag maxTag
      block 반복:
        entityDim entityTag elementType numElementsInBlock
        elemTag node1 node2 ...
    return list of tuples:
      (elem_id, elem_type, conn_node_ids, entity_dim, entity_tag)
    """
    if not elements_lines:
        raise ValueError("$Elements 섹션이 비어 있습니다.")

    num_blocks, total_elems, min_tag, max_tag = map(int, elements_lines[0].split())
    idx = 1

    elements: List[Tuple[int, int, List[int], int, int]] = []
    for _ in range(num_blocks):
        ent_dim, ent_tag, elem_type, nblock = map(int, elements_lines[idx].split())
        idx += 1
        for _k in range(nblock):
            parts = list(map(int, elements_lines[idx].split()))
            idx += 1
            elem_id = parts[0]
            conn = parts[1:]
            elements.append((elem_id, elem_type, conn, ent_dim, ent_tag))

    # elem_id 정렬
    elements.sort(key=lambda t: t[0])
    return elements


def attach_bc_node_tags_from_elements(bcs: List[BCInfo], elements: List[Tuple[int, int, List[int], int, int]]) -> None:
    """
    12) data_elements에서 BC의 (dim, entity_tag)에 해당하는 element들을 찾고,
        그 element들의 node tag를 유니크하게 모아서 bc.node_tags에 저장
    """
    # (ent_dim, ent_tag) -> set(node_ids)
    bucket: Dict[Tuple[int, int], Set[int]] = {}

    for (elem_id, elem_type, conn, ent_dim, ent_tag) in elements:
        key = (ent_dim, ent_tag)
        if key not in bucket:
            bucket[key] = set()
        bucket[key].update(conn)

    for bc in bcs:
        if bc.entity_tag is None:
            bc.node_tags = []
            continue
        key = (bc.dim, bc.entity_tag)
        nodes = sorted(bucket.get(key, set()))
        bc.node_tags = nodes


# ----------------------------
# 13~14: info_nodes / info_elements 생성
# ----------------------------
def build_info_nodes(node_ids: np.ndarray, xyz: np.ndarray) -> np.ndarray:
    """
    13) node id / x / y / z 형태로 (N,4) 행렬 생성
    """
    return np.column_stack([node_ids.astype(float), xyz])


def build_info_elements(elements: List[Tuple[int, int, List[int], int, int]]) -> Tuple[np.ndarray, np.ndarray]:
    """
    14) element id / element type / connectivity 를 '행렬 데이터'로 구성
    - elem_meta: (E,2) = [elem_id, elem_type]
    - conn_padded: (E,max_nodes) connectivity를 -1 padding
    """
    E = len(elements)
    elem_ids = np.array([e[0] for e in elements], dtype=int)
    elem_types = np.array([e[1] for e in elements], dtype=int)
    elem_meta = np.column_stack([elem_ids, elem_types])

    max_nodes = max((len(e[2]) for e in elements), default=0)
    conn_padded = -np.ones((E, max_nodes), dtype=int)
    for i, (_eid, _etype, conn, _d, _t) in enumerate(elements):
        conn_padded[i, :len(conn)] = conn

    return elem_meta, conn_padded


# ----------------------------
# 전체 파이프라인 (네 1~14를 실행)
# ----------------------------
def run_pipeline(path_msh: str) -> Dict[str, Any]:
    # 1~7
    remaining, dict_section = split_msh_sections_to_dict(path_msh)

    # 8
    bcs = parse_physicalnames_for_bcs(
        dict_section.get("PhysicalNames", []),
        bc_names=("displacementBC", "loadBC"),
    )

    # 9
    ent_phys_map = parse_entities_physical_map(dict_section.get("Entities", []))
    attach_entity_tags_to_bcs(bcs, ent_phys_map)

    # 10~11
    data_nodes = dict_section.get("Nodes", [])
    data_elements = dict_section.get("Elements", [])

    node_ids, xyz = parse_nodes_msh4(data_nodes)
    elements = parse_elements_msh4(data_elements)

    # 12
    attach_bc_node_tags_from_elements(bcs, elements)

    # 13
    info_nodes = build_info_nodes(node_ids, xyz)

    # 14
    info_elem_meta, info_elem_conn = build_info_elements(elements)

    # 결과 정리
    data_BC = [
        {
            "name": bc.name,
            "dim": bc.dim,
            "physical_tag": bc.physical_tag,
            "entity_tag": bc.entity_tag,
            "node_tags": bc.node_tags,
        }
        for bc in bcs
    ]

    return {
        "dict_section": dict_section,
        "data_BC": data_BC,
        "info_nodes": info_nodes,                 # (N,4): [node_id, x, y, z]
        "info_elements_meta": info_elem_meta,     # (E,2): [elem_id, elem_type]
        "info_elements_conn": info_elem_conn,     # (E,max_nodes): connectivity padded with -1
        "remaining_text_lines": remaining,         # 섹션 다 뽑고 남은 텍스트(보통 빈 리스트)
    }


# # ----------------------------
# # 실행 예시
# # ----------------------------
# if __name__ == "__main__":
#     path = "C:\\Users\\opew4\\Desktop\\Files_FreeCAD\\square_hex.msh"  # 네 파일 경로
#     out = run_pipeline(path)

#     print("info_nodes shape:", out["info_nodes"].shape)
#     print("elements meta shape:", out["info_elements_meta"].shape)
#     print("elements conn shape:", out["info_elements_conn"].shape)
#     print("BC summary:")
#     for bc in out["data_BC"]:
#         print(bc["name"], "dim=", bc["dim"], "phys=", bc["physical_tag"],
#               "entity=", bc["entity_tag"], "num_nodes=", len(bc["node_tags"] or []))


In [32]:
path = "C:\\Users\\opew4\\Desktop\\Files_FreeCAD\\square_hex.msh"  # .msh 파일 경로
out = run_pipeline(path)

print("info_nodes shape:", out["info_nodes"].shape)
print("elements meta shape:", out["info_elements_meta"].shape)
print("elements conn shape:", out["info_elements_conn"].shape)
print("BC summary:")

info_nodes shape: (1836, 4)
elements meta shape: (1300, 2)
elements conn shape: (1300, 8)
BC summary:


In [33]:
for bc in out["data_BC"]:
    print(bc["name"], "dim=", bc["dim"], "phys=", bc["physical_tag"],
            "entity=", bc["entity_tag"], "num_nodes=", len(bc["node_tags"] or []))

displacementBC dim= 2 phys= 2 entity= 1 num_nodes= 36
loadBC dim= 2 phys= 3 entity= 6 num_nodes= 36


In [34]:
# elements, nodes 정보 추출
info_elements_meta = out['info_elements_meta']
info_elements_conn = out['info_elements_conn']
info_nodes = out['info_nodes']

In [35]:
# hexahedral elements tag 정보 추출
info_elements_meta
info_elements_hexa_meta = info_elements_meta[info_elements_meta[:, 1] == 5]
tag_elements_hexa = info_elements_hexa_meta[:, 0]

In [36]:
# hexahedral elements의 node connectivity 정보 추출
conn_elements_hexa = info_elements_conn[tag_elements_hexa[0] - 1:]
conn_elements_hexa

array([[   1,    9,  237, ...,  253, 1053,  988],
       [  25,  253, 1053, ...,  254, 1054,  989],
       [  26,  254, 1054, ...,  255, 1055,  990],
       ...,
       [1834,  642,  173, ...,  643,  174,  692],
       [1835,  643,  174, ...,  644,  175,  693],
       [1836,  644,  175, ...,  179,    7,  229]], shape=(1250, 8))

In [37]:
# Gmsh Hex8 node signs in the order [1..8] = [0..7]
sxi  = np.array([-1, +1, +1, -1, -1, +1, +1, -1], dtype=float)
seta = np.array([-1, -1, +1, +1, -1, -1, +1, +1], dtype=float)
szet = np.array([-1, -1, -1, -1, +1, +1, +1, +1], dtype=float)

# returns dN/dxi, dN/deta, dN/dzeta as a (3,8) array
def dN_dxi_eta_zeta(xi, eta, zeta):
    dN_dxi   = 0.125 * sxi  * (1 + seta*eta) * (1 + szet*zeta)
    dN_deta  = 0.125 * seta * (1 + sxi*xi)   * (1 + szet*zeta)
    dN_dzeta = 0.125 * szet * (1 + sxi*xi)   * (1 + seta*eta)
    return np.vstack([dN_dxi, dN_deta, dN_dzeta])  # (3,8)

In [38]:
# ---------------------- hexahedral element별 stiffness matrix 계산
# Linear elasticity tensor (6x6)
C = np.zeros((6, 6))

# Isotropic material properties
E = 210e3  # Young's modulus N/mm2
nu = 0.3   # Poisson's ratio

# Lame parameters
lam = E * nu / ((1 + nu) * (1 - 2 * nu))
mu  = E / (2 * (1 + nu))   # shear modulus

# Normal components
C[0, 0] = lam + 2 * mu
C[1, 1] = lam + 2 * mu
C[2, 2] = lam + 2 * mu

C[0, 1] = lam
C[0, 2] = lam
C[1, 0] = lam
C[1, 2] = lam
C[2, 0] = lam
C[2, 1] = lam

# Shear components
C[3, 3] = mu   # yz
C[4, 4] = mu   # xz
C[5, 5] = mu   # xy

# Gauss 점과 가중치
xi_g = [-0.5773502691896257, 0.5773502691896257]
eta_g = [-0.5773502691896257, 0.5773502691896257]
zeta_g = [-0.5773502691896257, 0.5773502691896257]
w_g = [1.0, 1.0]

# Global stiffness matrix 초기화
ndof = info_nodes.shape[0] * 3
Kg = lil_matrix((ndof, ndof))

for e_hex, tag_nodes_hex in enumerate(conn_elements_hexa):
    tag_e_hex = e_hex + tag_elements_hexa[0]

    # element별 보유 node coordinates 출력
    node_coords_hex = []
    for tag_node in tag_nodes_hex:
        node_coords_hex.append(info_nodes[tag_node - 1, 1:])
    node_coords_hex = np.array(node_coords_hex)

    Ke = np.zeros((3*8, 3*8))  # element stiffness matrix 초기화
    # Gauss 점 하나씩 순회
    for i_g, xi in enumerate(xi_g):
        for j_g, eta in enumerate(eta_g):
            for k_g, zeta in enumerate(zeta_g):
                w = w_g[i_g] * w_g[j_g] * w_g[k_g]

                # shape function 미분값 계산
                dN_dxi_g = dN_dxi_eta_zeta(xi, eta, zeta)

                # Jacobian 행렬 계산
                J_g = dN_dxi_g @ node_coords_hex

                # Jacobian 행렬의 역행렬 계산
                J_inv_g = np.linalg.inv(J_g)

                # dN/dx(@gauss point) 계산
                dN_dx_g = J_inv_g @ dN_dxi_g

                # B matrix 계산
                B_g = np.zeros((6, 3*8))
                for a in range(8):
                    dN_dx = dN_dx_g[0, a]
                    dN_dy = dN_dx_g[1, a]
                    dN_dz = dN_dx_g[2, a]
                    B_g[0, 3*a + 0] = dN_dx
                    B_g[1, 3*a + 1] = dN_dy
                    B_g[2, 3*a + 2] = dN_dz
                    B_g[3, 3*a + 1] = dN_dz; B_g[3, 3*a + 2] = dN_dy
                    B_g[4, 3*a + 0] = dN_dz; B_g[4, 3*a + 2] = dN_dx
                    B_g[5, 3*a + 0] = dN_dy; B_g[5, 3*a + 1] = dN_dx
                
                detJ = np.linalg.det(J_g)
                if detJ <= 0:
                    print("WARNING: detJ <= 0", tag_e_hex, detJ, tag_nodes_hex)

                # Ke(@gauss point) 계산
                Ke += w * B_g.transpose() @ C @ B_g * np.linalg.det(J_g)
    
    # Global stiffness matrix에 Ke 추가
    for a_local, tag_node_a in enumerate(tag_nodes_hex):
        for b_local, tag_node_b in enumerate(tag_nodes_hex):
            for i in range(3):
                for j in range(3):
                    Kg[(tag_node_a - 1)*3 + i, (tag_node_b - 1)*3 + j] += Ke[a_local*3 + i, b_local*3 + j]

In [39]:
d = Kg.diagonal()
np.min(d)

np.float64(493589.74358974345)

In [40]:
# ----------------------------
# 1) BC 노드 태그 가져오기
# ----------------------------
# data_BC는 네 스크린샷처럼 list[dict] 형태라고 가정
disp_nodes = None
load_nodes = None

data_BC = out['data_BC']

for bc in data_BC:
    if bc["name"] == "displacementBC":
        disp_nodes = np.array(bc["node_tags"], dtype=int)
    elif bc["name"] == "loadBC":
        load_nodes = np.array(bc["node_tags"], dtype=int)

if disp_nodes is None or load_nodes is None:
    raise ValueError("displacementBC 또는 loadBC를 data_BC에서 찾지 못했습니다.")

ndof = info_nodes.shape[0] * 3

# ----------------------------
# 2) Force vector 만들기
# ----------------------------
F = np.zeros(ndof)

# 총 하중 (N). 50x50x500 cantilever 기준으로 10kN이면 꽤 큰 하중이지만 변형은 과도하진 않음(선형 범위 가정).
Fy_total = -1.0e4  # -10,000 N (아래방향)

fy_each = Fy_total / len(load_nodes)

for tag_node in load_nodes:
    dof_y = (tag_node - 1) * 3 + 1  # y방향 DOF: [x=0, y=1, z=2]
    F[dof_y] += fy_each

# ----------------------------
# 3) Dirichlet BC(u=v=w=0) 적용
#    (행/열 0 만들고 대각 1 세팅)
# ----------------------------
# Kg는 지금 lil_matrix로 이미 만들었으니 그대로 수정 가능.
# 혹시 csr이면 lil로 바꾸고 진행:
if not isinstance(Kg, lil_matrix):
    Kg = Kg.tolil()

fixed_dofs = []
for tag_node in disp_nodes:
    base = (tag_node - 1) * 3
    fixed_dofs.extend([base + 0, base + 1, base + 2])  # u,v,w = 0

fixed_dofs = np.array(fixed_dofs, dtype=int)

# u=0이므로 RHS 조정은 필요 없음(일반값이면 F -= K[:,dof]*u0 해야 함)
for dof in fixed_dofs:
    Kg.rows[dof] = [dof] # dof 번째 행과 연결된 열 중에서 dof 번째 열만 남기고 나머지 열은 제거(즉 대각 성분만 남김)
    Kg.data[dof] = [1.0] # dof번째 대각 성분을 1로 설정
    F[dof] = 0.0 # RHS도 0으로 설정(부여된 변위가 0이므로)

# 열(column)도 0으로 만들어 symmetry 유지
# (LIL에서 열 제거는 직접 루프가 필요)
fixed_set = set(fixed_dofs.tolist())
for i in range(ndof):
    if i in fixed_set: # 이미 대각 성분만 남긴 행은 건너뜀
        continue
    
    # 아직 처리 안 한 행에 대해 열 제거 진행
    row_cols = Kg.rows[i] # 아직 처리 안 한 i번째 행의 열 인덱스 리스트
    row_data = Kg.data[i] # 아직 처리 안 한 i번째 행의 데이터 리스트
    
    # fixed_dofs에 해당하는 열 성분 제거
    new_cols = []
    new_data = []
    for col, value in zip(row_cols, row_data):
        if col in fixed_set:
            continue
        new_cols.append(col)
        new_data.append(value)
    Kg.rows[i] = new_cols # i번째 행의 열을 fixed_dofs 제거한 새 열 리스트로 교체
    Kg.data[i] = new_data # i번째 행의 데이터를 fixed_dofs 제거한 새 데이터 리스트로 교체

# solver용 CSR 변환
K_csr = Kg.tocsr()

# ----------------------------
# 4) 해 풀기: K d = F
# ----------------------------
d = spsolve(K_csr, F)

# d는 (ndof,) 변위 벡터. 필요하면 (nnode,3)으로 reshape:
disp = d.reshape((-1, 3))

In [41]:
disp

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-1.77820908e-04, -3.38770812e+00,  1.66052202e-01],
       [-1.08090051e-04, -3.49915242e+00,  1.66377325e-01],
       [-3.42769658e-05, -3.61080641e+00,  1.66547550e-01]],
      shape=(1836, 3))

In [42]:
# disp: (nnode,3) or (ndof,) -> (nnode,3)
if disp.ndim == 1:
    disp_vec = disp.reshape((-1, 3))
else:
    disp_vec = disp

gmsh_path = "square_hex.msh" # 원본 gmsh 메쉬 파일
mesh_unsolved_path = "mesh.vtu" # ParaView에서 열 파일
mesh_solved_path = "result.vtu" # ParaView에서 열 파일
mesh_solved_scaled_path = "result_scaled.vtu" # ParaView에서 열 파일

# ------------------ 원본 메쉬 vtu로 저장
mesh = meshio.read(gmsh_path)
meshio.write(mesh_unsolved_path, mesh)
print("Wrote:", mesh_unsolved_path)

# ------------------ 변형된 형상 스케일링 없이 저장
# mesh.points: (nnode,3)
# disp_vec:    (nnode,3)  <-- 이 둘의 노드 순서가 같아야 함
assert mesh.points.shape[0] == disp_vec.shape[0], \
    f"node mismatch: mesh has {mesh.points.shape[0]} nodes, disp has {disp_vec.shape[0]}"

mesh.point_data = mesh.point_data or {}
mesh.point_data["disp"] = disp_vec               # 벡터 필드
mesh.point_data["disp_mag"] = np.linalg.norm(disp_vec, axis=1)  # 크기 스칼라

meshio.write(mesh_solved_path, mesh)
print("Wrote:", mesh_solved_path)

# ------------------ 변형된 형상에 스케일링 적용하여 저장
scale = 10.0  # mm 단위면 1.0 그대로, 보기용이면 10~100도 가능
mesh_def = meshio.Mesh(
    points=mesh.points + scale * disp_vec,
    cells=mesh.cells,
    point_data={"disp": disp_vec, "disp_mag": np.linalg.norm(disp_vec, axis=1)},
    cell_data=mesh.cell_data
)
meshio.write(mesh_solved_scaled_path, mesh_def)
print("Wrote:", mesh_solved_scaled_path)

Info: VTU format cannot write cell_sets. Converting them to cell_data...

Wrote: mesh.vtu
Wrote: result.vtu
Wrote: result_scaled.vtu


In [45]:
mesh.point_data['disp']

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-1.77820908e-04, -3.38770812e+00,  1.66052202e-01],
       [-1.08090051e-04, -3.49915242e+00,  1.66377325e-01],
       [-3.42769658e-05, -3.61080641e+00,  1.66547550e-01]],
      shape=(1836, 3))